In [1]:
import sys, os

# Place the root of the python environment
env_root = '/N/project/baby_vision_benchmark/envs/bvbenv/lib/python3.10/site-packages/'
sys.path.insert(0, env_root)


In [2]:
from utils import load_model

model = load_model('vit_s_none')

{'num_frames': 16, 't_patch_size': 2, 'img_size': 224, 'patch_size': 14, 'in_chans': 3, 'num_classes': None, 'depth': 32, 'qk_scale': None, 'drop_rate': 0.0, 'attn_drop_rate': 0.0, 'drop_path_rate': 0.1, 'dropout': 0.3, 'sep_pos_embed': True, 'cls_embed': True, 'kwargs': {}, 'embed_dim': 1280, 'mlp_ratio': 4, 'no_qkv_bias': False, 'norm_layer': functools.partial(<class 'torch.nn.modules.normalization.LayerNorm'>, eps=1e-06), 'num_heads': 16, 'self': VisionTransformer(), '__class__': <class 'models_vit.VisionTransformer'>}
img_size (224, 224) patch_size (14, 14) frames 16 t_patch_size 2
Loaded with message: _IncompatibleKeys(missing_keys=[], unexpected_keys=['decoder_cls_token', 'mask_token', 'decoder_pos_embed_spatial', 'decoder_pos_embed_temporal', 'decoder_pos_embed_class', 'decoder_embed.weight', 'decoder_embed.bias', 'decoder_blocks.0.norm1.weight', 'decoder_blocks.0.norm1.bias', 'decoder_blocks.0.attn.q.weight', 'decoder_blocks.0.attn.q.bias', 'decoder_blocks.0.attn.k.weight', 'de

In [3]:
sys.path

['/N/project/baby_vision_benchmark/envs/bvbenv/lib/python3.10/site-packages/',
 '/N/project/baby_vision_benchmark/github/saycam-video-models',
 '/N/soft/sles15/deeplearning/Python-3.10.5/lib/python310.zip',
 '/N/soft/sles15/deeplearning/Python-3.10.5/lib/python3.10',
 '/N/soft/sles15/deeplearning/Python-3.10.5/lib/python3.10/lib-dynload',
 '',
 '/N/soft/sles15/deeplearning/Python-3.10.5/lib/python3.10/site-packages']

In [3]:
model

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv3d(3, 1280, kernel_size=(2, 14, 14), stride=(2, 14, 14))
  )
  (blocks): ModuleList(
    (0): Block(
      (norm1): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (q): Linear(in_features=1280, out_features=1280, bias=True)
        (k): Linear(in_features=1280, out_features=1280, bias=True)
        (v): Linear(in_features=1280, out_features=1280, bias=True)
        (proj): Linear(in_features=1280, out_features=1280, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=1280, out_features=5120, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (fc2): Linear(in_features=5120, out_features=1280, bias=True)
        (drop2): Dropout(p=0.0, in

In [4]:
import torch

In [5]:
# input shape: Batch, Channel, Time, Width, Height
xtt = torch.rand(1,3,16,224,224)

In [6]:
ytt = model(xtt)

In [7]:
ytt.shape

torch.Size([1, 1280])

We can replace head:Identity with a Linear layer.

We'd need to 

In [8]:
import torch
from PIL import Image
from torchvision import transforms

def image_to_video_tensor(filepath, time_steps, transform=None):
    # Open the image file
    image = Image.open(filepath).convert("RGB")
    
    # Apply the optional transform function if provided
    if transform is not None:
        image = transform(image)
    
    # Check if the image is already a tensor, if not, convert it to a tensor
    if not isinstance(image, torch.Tensor):
        image = transforms.ToTensor()(image)
    c,h,w = image.shape
    
    # Repeat the tensor across the time dimension
    video_tensor = image.unsqueeze(0).unsqueeze(2).repeat(1, 1, time_steps, 1, 1)
    
    return video_tensor

In [9]:
from torchvision import transforms

filepath = "/N/project/baby_vision_benchmark/dataset1/individual_objects/realistic/CarSide_v1.jpg"
time_steps = 16

# Define the transform function
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.25, 0.25, 0.25))
])

video_tensor = image_to_video_tensor(filepath, time_steps, transform)
print(video_tensor.shape)  # Output: (1, 3, 16, 224, 224)

torch.Size([1, 3, 16, 224, 224])


In [10]:
ytt = model(video_tensor)
ytt.shape

torch.Size([1, 1280])